# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'10-23-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'10-22-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-10-23 04:24:46,33.93911,67.709953,40626,1505,33831,5290.0,Afghanistan,104.360985,3.704524
1,NaN,NaN,NaN,Albania,2020-10-23 04:24:46,41.15330,20.168300,18250,465,10395,7390.0,Albania,634.164987,2.547945
2,NaN,NaN,NaN,Algeria,2020-10-23 04:24:46,28.03390,1.659600,55357,1888,38618,14851.0,Algeria,126.238731,3.410589
3,NaN,NaN,NaN,Andorra,2020-10-23 04:24:46,42.50630,1.521800,3811,63,2470,1278.0,Andorra,4932.375591,1.653109
4,NaN,NaN,NaN,Angola,2020-10-23 04:24:46,-11.20270,17.873900,8582,260,3305,5017.0,Angola,26.111879,3.029597


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,39994,40026,40073,40141,40200,40287,40357,40510,40626,40687
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,15955,16212,16501,16774,17055,17350,17651,17948,18250,18556
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,53584,53777,53998,54203,54402,54616,54829,55081,55357,55630


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1481,1481,1485,1488,1492,1497,1499,1501,1505,1507
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,434,439,443,448,451,454,458,462,465,469
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1827,1827,1841,1846,1856,1865,1873,1880,1888,1897


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,33354,33447,33516,33561,33614,33760,33790,33824,33831,34010
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,9762,9864,9957,10001,10071,10167,10225,10341,10395,10466
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,37603,37603,37856,37971,38088,38215,38346,38482,38618,38788


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1928,1949,1966,1983,1989,1999,2010,2021,2023,2030
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,6270,6285,6333,6350,6369,6375,6405,6443,6475,6615
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,950,965,968,977,981,981,988,996,997,1012


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,28,28,28,28,28,28,29,30,30,30
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,66,66,67,67,67,67,69,69,69,69
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,9,9,9,9,9


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
633,1001.0,Autauga,Alabama,US,2020-10-23 04:24:46,32.539527,-86.644082,2023,30,0,1993.0,"Autauga, Alabama, US",3620.970485,1.482946
670,1075.0,Lamar,Alabama,US,2020-10-23 04:24:46,33.779950,-88.096680,466,5,0,461.0,"Lamar, Alabama, US",3375.588555,1.072961
671,1077.0,Lauderdale,Alabama,US,2020-10-23 04:24:46,34.901719,-87.656247,2148,41,0,2107.0,"Lauderdale, Alabama, US",2316.427439,1.908752


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,8407702,223032,3353056
India,7761312,117306,6948497
Brazil,5323630,155900,4526393


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,8407702,223032,3353056,4831614,2020-10-23 04:24:46,37.936303,-91.379001
India,7761312,117306,6948497,695509,2020-10-23 04:24:46,23.088275,81.806127
Brazil,5323630,155900,4526393,641337,2020-10-23 04:24:46,-12.669522,-48.480493
Russia,1453923,25072,1102564,326287,2020-10-23 04:24:46,54.546312,62.120860
Argentina,1053650,27957,851854,173839,2020-10-23 04:24:46,-38.416100,-63.616700


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,893364,17256,0
Texas,871453,17659,0
Florida,768091,16267,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,893364,17256,0,876108,2020-10-23 04:24:46,37.843962,-120.728594
Texas,871453,17659,0,853794,2020-10-23 04:24:46,31.660643,-98.653069
Florida,768091,16267,0,751824,2020-10-23 04:24:46,28.940755,-82.700744
New York,490134,33396,0,456738,2020-10-23 04:24:46,42.544151,-75.474183
Illinois,363740,9647,0,354093,2020-10-23 04:24:46,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,294065,6956,0
Florida,Miami-Dade,180497,3585,0
Illinois,Cook,170039,5390,0
Texas,Harris,156742,2750,0
Arizona,Maricopa,152101,3536,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,294065,6956,0,287109,2020-10-23 04:24:46,34.308284,-118.228241,6037.0
Florida,Miami-Dade,180497,3585,0,176912,2020-10-23 04:24:46,25.611236,-80.551706,12086.0
Illinois,Cook,170039,5390,0,164649,2020-10-23 04:24:46,41.841448,-87.816588,17031.0
Texas,Harris,156742,2750,0,153992,2020-10-23 04:24:46,29.858649,-95.393395,48201.0
Arizona,Maricopa,152101,3536,0,148565,2020-10-23 04:24:46,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-21,40510,17948,55081,3811,8338,122,1037325,68530,27458,69409,...,792194,2663,64010,88035,1144,48628,10,2057,16000,8215
2020-10-22,40626,18250,55357,3811,8582,122,1053650,70836,27476,71844,...,813451,2701,64439,88416,1148,49134,10,2057,16035,8242
2020-10-23,40687,18556,55630,4038,8829,122,1069368,73310,27495,74415,...,834010,2759,64724,88718,1148,49579,10,2060,16095,8257


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-21,1501,462,1880,63,255,3,27519,1121,905,925,...,44248,53,534,753,35,427,1,597,346,236
2020-10-22,1505,465,1888,63,260,3,27957,1131,905,941,...,44437,53,540,759,35,435,1,597,346,236
2020-10-23,1507,469,1897,69,265,3,28338,1145,905,954,...,44661,53,542,764,35,439,1,599,346,236


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-21,33824,10341,38482,2470,3040,101,840520,49219,25148,52617,...,2636,2172,61068,81626,1046,41935,8,1344,15168,7725
2020-10-22,33831,10395,38618,2470,3305,107,851854,49787,25161,53970,...,2652,2204,61658,82284,1049,42544,8,1344,15168,7742
2020-10-23,34010,10466,38788,2729,3384,107,866695,50276,25173,55195,...,2657,2241,61957,82899,1049,42850,8,1354,15179,7771


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1928,1949,1966,1983,1989,1999,2010,2021,2023,2030
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,426,429,441,446,455,457,456,459,466,474
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1950,1974,1994,2029,2043,2063,2087,2125,2148,2180
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,748,759,772,782,788,792,800,811,821,837
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,6346,6367,6400,6421,6430,6439,6449,6461,6496,6517


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-21,175674,11602,0,233912,101596,886939,88826,64871,23375,49,...,235861,868981,98006,1971,1337,168260,99874,20735,182687,9848
2020-10-22,177064,11835,0,234906,102798,893364,90199,65373,23528,49,...,237907,871453,99549,1987,1343,169566,100525,21057,186100,10119
2020-10-23,180916,12113,0,235882,104135,898799,91549,66052,23687,49,...,241513,878567,101509,2016,1346,170730,101345,21393,190478,10545


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-21        2021    6443     996  801   1873     637    998    4160   
2020-10-22        2023    6475     997  811   1893     637   1001    4189   
2020-10-23        2030    6615    1012  825   1911     639   1002    4224   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-21         1330      723  ...     481    106      577      154   
2020-10-22         1336      726  ...     504    113      598      157   
2020-10-23         1343      731  ...     536    115      634      158   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-21            436   715   407          0      142    103  
2020-10-22            443   727   410          0      143    115  
2020-10-23            451   740   425          0      144    121  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-21,2828,68,0,5854,1751,17167,2194,4567,670,0,...,2970,17652,557,58,21,3511,2286,414,1681,61
2020-10-22,2843,68,0,5859,1772,17256,2198,4569,670,0,...,3011,17659,563,58,21,3520,2289,420,1703,68
2020-10-23,2859,68,0,5865,1782,17305,2211,4577,678,0,...,3076,17773,567,58,21,3535,2296,424,1745,68


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-21          30      69       9   14     25      17     40      59   
2020-10-22          30      69       9   14     25      17     40      60   
2020-10-23          30      69       9   14     25      17     40      61   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-21           43       14  ...       3      1        4        1   
2020-10-22           43       14  ...       3      1        4        1   
2020-10-23           43       14  ...       3      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-21              2     1     2          0        7      0  
2020-10-22              2     1     2          7        7      0  
2020-10-23              2     1     3          0        7      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-21,0.003791,0.016826,0.004596,0.051891,0.035905,0.02521,0.017984,0.027529,0.000547,0.029028,...,0.034889,0.015250,0.002804,0.004461,0.002629,0.010368,0.0,0.000000,0.001126,0.003420
2020-10-22,0.002863,0.016826,0.005011,0.000000,0.029264,0.00000,0.015738,0.033649,0.000656,0.035082,...,0.026833,0.014270,0.006702,0.004328,0.003497,0.010406,0.0,0.000000,0.002188,0.003287
2020-10-23,0.001502,0.016767,0.004932,0.059564,0.028781,0.00000,0.014918,0.034926,0.000692,0.035786,...,0.025274,0.021474,0.004423,0.003416,0.000000,0.009057,0.0,0.001458,0.003742,0.001820


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-21,0.001334,0.008734,0.003737,0.016129,0.015936,0.0,0.015461,0.018165,0.0,0.012035,...,0.004335,0.019231,0.000000,0.008032,0.0,0.014252,0.0,0.00000,0.0,0.012876
2020-10-22,0.002665,0.006494,0.004255,0.000000,0.019608,0.0,0.015916,0.008921,0.0,0.017297,...,0.004271,0.000000,0.011236,0.007968,0.0,0.018735,0.0,0.00000,0.0,0.000000
2020-10-23,0.001329,0.008602,0.004767,0.095238,0.019231,0.0,0.013628,0.012378,0.0,0.013815,...,0.005041,0.000000,0.003704,0.006588,0.0,0.009195,0.0,0.00335,0.0,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-21,0.001006,0.011345,0.003547,0.086670,0.000988,0.000000,0.013106,0.009952,0.000915,0.023538,...,0.003808,0.014006,0.002594,0.016311,0.000000,0.011579,0.0,0.004484,0.008645,0.004290
2020-10-22,0.000207,0.005222,0.003534,0.000000,0.087171,0.059406,0.013485,0.011540,0.000517,0.025714,...,0.006070,0.014733,0.009661,0.008061,0.002868,0.014522,0.0,0.000000,0.000000,0.002201
2020-10-23,0.005291,0.006830,0.004402,0.104858,0.023903,0.000000,0.017422,0.009822,0.000477,0.022698,...,0.001885,0.016788,0.004849,0.007474,0.000000,0.007193,0.0,0.007440,0.000725,0.003746


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-21,0.006566,0.018523,NaN,0.004186,0.011499,0.003613,0.014470,0.006454,0.002144,0.0,...,0.009813,0.007775,0.014103,0.007669,0.000000,0.005919,0.007302,0.010478,0.023560,0.033802
2020-10-22,0.007912,0.020083,NaN,0.004249,0.011831,0.007244,0.015457,0.007738,0.006545,0.0,...,0.008675,0.002845,0.015744,0.008118,0.004488,0.007762,0.006518,0.015529,0.018682,0.027518
2020-10-23,0.021755,0.023490,NaN,0.004155,0.013006,0.006084,0.014967,0.010387,0.006758,0.0,...,0.015157,0.008163,0.019689,0.014595,0.002234,0.006865,0.008157,0.015957,0.023525,0.042099


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                   \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount  Bullock   
2020-10-21      0.005473  0.005933  0.008097  0.012642  0.013528  0.00315   
2020-10-22      0.000990  0.004967  0.001004  0.012484  0.010678  0.00000   
2020-10-23      0.003460  0.021622  0.015045  0.017263  0.009509  0.00314   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-21      0.000000  0.007996  0.022291  0.001385  ...  0.041126   
2020-10-22      0.003006  0.006971  0.004511  0.004149  ...  0.047817   
2020-10-23      0.000999  0.008355  0.005240  0.006887  ...  0.063492   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-21      0.081633  0.070501  0.033557   0.018692 -0.001397  0.007426   
2020-10-22      0.066038  0.036395  0.019481   0.016055  0.016783  0.007371   
2020-10-23      0.017699  0.060201  0.006369   0.018059  0.017882  0.036585   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-21            NaN  0.021583  0.019802  
2020-10-22            NaN  0.007042  0.116505  
2020-10-23            NaN  0.006993  0.052174  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-21,0.008200,0.014925,NaN,0.002912,0.013310,0.006567,0.005500,0.001755,0.002994,NaN,...,0.006098,0.005354,0.010889,0.0,0.0,0.008329,0.001753,0.007299,0.029394,0.000000
2020-10-22,0.005304,0.000000,NaN,0.000854,0.011993,0.005184,0.001823,0.000438,0.000000,NaN,...,0.013805,0.000397,0.010772,0.0,0.0,0.002563,0.001312,0.014493,0.013087,0.114754
2020-10-23,0.005628,0.000000,NaN,0.001024,0.005643,0.002840,0.005914,0.001751,0.011940,NaN,...,0.021588,0.006456,0.007105,0.0,0.0,0.004261,0.003058,0.009524,0.024662,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                \
Admin2           Autauga Baldwin Barbour Bibb    Blount Bullock Butler   
2020-10-21      0.034483     0.0     0.0  0.0  0.041667     0.0    0.0   
2020-10-22      0.000000     0.0     0.0  0.0  0.000000     0.0    0.0   
2020-10-23      0.000000     0.0     0.0  0.0  0.000000     0.0    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-10-21      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2020-10-22      0.016949      0.0      0.0  ...     0.0    0.0      0.0   
2020-10-23      0.016667      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-21          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-22          0.0        0.0   0.0   0.0        inf      0.0    NaN  
2020-10-23          0.0        0.0   0.0   0.5       -1.0      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-21,0.002783,0.016976,0.004229,0.036339,0.031510,0.014595,0.016349,0.023101,0.000696,0.025672,...,0.030907,0.017042,0.003986,0.005738,0.002749,0.010150,2.089768e-38,0.000169,0.002802,0.003161
2020-10-22,0.002823,0.016901,0.004620,0.018170,0.030387,0.007298,0.016043,0.028375,0.000676,0.030377,...,0.028870,0.015656,0.005344,0.005033,0.003123,0.010278,1.044884e-38,0.000084,0.002495,0.003224
2020-10-23,0.002162,0.016834,0.004776,0.038867,0.029584,0.003649,0.015480,0.031650,0.000684,0.033081,...,0.027072,0.018565,0.004884,0.004224,0.001561,0.009667,5.224419e-39,0.000771,0.003118,0.002522


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-21,0.001700,0.008491,0.004127,0.014596,0.014564,6.422152e-58,0.014792,0.013927,0.000081,0.011058,...,0.004071,0.014542,0.003486,0.008670,7.064621e-17,0.014703,0.0,0.000107,0.000047,0.008075
2020-10-22,0.002183,0.007492,0.004191,0.007298,0.017086,3.211076e-58,0.015354,0.011424,0.000041,0.014178,...,0.004171,0.007271,0.007361,0.008319,3.532311e-17,0.016719,0.0,0.000054,0.000023,0.004037
2020-10-23,0.001756,0.008047,0.004479,0.051268,0.018158,1.605538e-58,0.014491,0.011901,0.000020,0.013996,...,0.004606,0.003636,0.005532,0.007453,1.766155e-17,0.012957,0.0,0.001702,0.000012,0.002019


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-21,0.001481,0.009167,0.003464,0.057057,0.002182,0.000434,0.016584,0.008724,0.000663,0.021336,...,0.004911,0.012484,0.004234,0.011309,0.001858,0.011544,1.224473e-40,0.002430,0.005108,0.002774
2020-10-22,0.000844,0.007195,0.003499,0.028529,0.044677,0.029920,0.015034,0.010132,0.000590,0.023525,...,0.005490,0.013608,0.006948,0.009685,0.002363,0.013033,6.122366e-41,0.001215,0.002554,0.002487
2020-10-23,0.003067,0.007012,0.003951,0.066693,0.034290,0.014960,0.016228,0.009977,0.000533,0.023112,...,0.003688,0.015198,0.005899,0.008580,0.001181,0.010113,3.061183e-41,0.004328,0.001640,0.003117


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-21,0.006204,0.018725,NaN,0.004053,0.009574,0.003884,0.013862,0.007558,0.003912,4.040667e-67,...,0.009624,0.006671,0.013106,0.006419,0.000526,0.005605,0.006306,0.011201,0.024883,0.029582
2020-10-22,0.007058,0.019404,NaN,0.004151,0.010702,0.005564,0.014659,0.007648,0.005229,2.020334e-67,...,0.009150,0.004758,0.014425,0.007268,0.002507,0.006683,0.006412,0.013365,0.021783,0.028550
2020-10-23,0.014406,0.021447,NaN,0.004153,0.011854,0.005824,0.014813,0.009017,0.005993,1.010167e-67,...,0.012153,0.006461,0.017057,0.010932,0.002370,0.006774,0.007285,0.014661,0.022654,0.035325


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-21      0.005451  0.004697  0.006566  0.008975  0.010254  0.002014   
2020-10-22      0.003221  0.004832  0.003785  0.010730  0.010466  0.001007   
2020-10-23      0.003340  0.013227  0.009415  0.013996  0.009987  0.002073   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-21      0.000888  0.007163  0.017497  0.004180  ...  0.038172   
2020-10-22      0.001947  0.007067  0.011004  0.004165  ...  0.042995   
2020-10-23      0.001473  0.007711  0.008122  0.005526  ...  0.053243   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-21      0.084119  0.053597  0.027255   0.015974  0.003950  0.005929   
2020-10-22      0.075078  0.044996  0.023368   0.016015  0.010367  0.006650   
2020-10-23      0.046389  0.052598  0.014869   0.017037  0.014124  0.021618   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-21      -0.999999  0.019905  0.032477  
2020-10-22      -0.999999  0.013474  0.074491  
2020-10-23      -0.999999  0.010233  0.063333  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-21,0.005979,0.008561,NaN,0.002026,0.010882,0.004700,0.003396,0.001499,0.002679,NaN,...,0.006858,0.004265,0.009271,1.170187e-25,0.000391,0.007524,0.004719,0.009848,0.022347,0.017654
2020-10-22,0.005642,0.004280,NaN,0.001440,0.011438,0.004942,0.002610,0.000968,0.001340,NaN,...,0.010332,0.002331,0.010021,5.850936e-26,0.000195,0.005044,0.003016,0.012170,0.017717,0.066204
2020-10-23,0.005635,0.002140,NaN,0.001232,0.008540,0.003891,0.004262,0.001360,0.006640,NaN,...,0.015960,0.004393,0.008563,2.925468e-26,0.000098,0.004653,0.003037,0.010847,0.021190,0.033102


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-21      0.026172  0.007773  0.000017  0.019568  0.033414  0.000489   
2020-10-22      0.013086  0.003887  0.000009  0.009784  0.016707  0.000245   
2020-10-23      0.006543  0.001943  0.000004  0.004892  0.008353  0.000122   

Province_State                                                  ...  Wyoming  \
Admin2                Butler   Calhoun      Chambers  Cherokee  ...     Park   
2020-10-21      3.106059e-09  0.000928  7.266097e-07  0.000300  ...  0.12500   
2020-10-22      1.553029e-09  0.008939  3.633049e-07  0.000150  ...  0.06250   
2020-10-23      7.765147e-10  0.012803  1.816524e-07  0.000075  ...  0.03125   

Province_State                                                       \
Admin2         Platte      Sheridan Sublette Sweetwater Teton Uinta   
2020-10-21        0.0  4.365575e-11      0.0        0.0   0.0  0.00   
2020-10-22        0.0  2.182787e-11      0.0        0.0   0.0  0.00   
2020-10-23        0.0  1.091394e-11      0.0        0.0   0.0  0.25   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2020-10-21      -0.533333  0.000326    NaN  
2020-10-22      -0.533333  0.000163    NaN  
2020-10-23      -0.999573  0.000081    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201023,AK,13161,NaN,539585,NaN,totalTestsViral,552746,59.0,NaN,...,552746,0,0,1b73fb16c097754ce2024af0e49f9d9514248af0,0,0,0,0,0,NaN
1,20201023,AL,180916,25001.0,1133858,NaN,totalTestsViral,1289773,888.0,19595.0,...,1314774,16,147,5323772280706f0dd467c842b6640d6f931d3c5a,0,0,0,0,0,NaN
2,20201023,AR,104135,6796.0,1170162,NaN,totalTestsViral,1267501,610.0,6678.0,...,1274297,10,152,678aecb11580eec0d6282ac63a5438e7d88d82c1,0,0,0,0,0,NaN
3,20201023,AS,0,NaN,1616,NaN,totalTestsViral,1616,NaN,NaN,...,1616,0,0,48257900fad58cf22cdad9759edd7311027a9b3b,0,0,0,0,0,NaN
4,20201023,AZ,235882,5394.0,1450981,NaN,totalTestsPeopleViral,1681469,815.0,20967.0,...,1686863,6,29,fa8d4a28172c518fd0235a20732b7759a3c70bdd,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201023,AK,13161.0,NaN,539585.0,NaN,totalTestsViral,552746.0,59.0,NaN,...,552746,0,0,1b73fb16c097754ce2024af0e49f9d9514248af0,0,0,0,0,0,NaN
1,20201023,AL,180916.0,25001.0,1133858.0,NaN,totalTestsViral,1289773.0,888.0,19595.0,...,1314774,16,147,5323772280706f0dd467c842b6640d6f931d3c5a,0,0,0,0,0,NaN
2,20201023,AR,104135.0,6796.0,1170162.0,NaN,totalTestsViral,1267501.0,610.0,6678.0,...,1274297,10,152,678aecb11580eec0d6282ac63a5438e7d88d82c1,0,0,0,0,0,NaN
3,20201023,AS,0.0,NaN,1616.0,NaN,totalTestsViral,1616.0,NaN,NaN,...,1616,0,0,48257900fad58cf22cdad9759edd7311027a9b3b,0,0,0,0,0,NaN
4,20201023,AZ,235882.0,5394.0,1450981.0,NaN,totalTestsPeopleViral,1681469.0,815.0,20967.0,...,1686863,6,29,fa8d4a28172c518fd0235a20732b7759a3c70bdd,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-23,AK,13161,NaN,539585,NaN,totalTestsViral,552746,59.0,NaN,NaN,...,552746,0,0,1b73fb16c097754ce2024af0e49f9d9514248af0,0,0,0,0,0,NaN
2020-10-23,AL,180916,25001.0,1133858,NaN,totalTestsViral,1289773,888.0,19595.0,NaN,...,1314774,16,147,5323772280706f0dd467c842b6640d6f931d3c5a,0,0,0,0,0,NaN
2020-10-23,AR,104135,6796.0,1170162,NaN,totalTestsViral,1267501,610.0,6678.0,237.0,...,1274297,10,152,678aecb11580eec0d6282ac63a5438e7d88d82c1,0,0,0,0,0,NaN
2020-10-23,AS,0,NaN,1616,NaN,totalTestsViral,1616,NaN,NaN,NaN,...,1616,0,0,48257900fad58cf22cdad9759edd7311027a9b3b,0,0,0,0,0,NaN
2020-10-23,AZ,235882,5394.0,1450981,NaN,totalTestsPeopleViral,1681469,815.0,20967.0,172.0,...,1686863,6,29,fa8d4a28172c518fd0235a20732b7759a3c70bdd,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-23,AK,13161.0,NaN,539585.0,NaN,totalTestsViral,552746.0,59.0,NaN,NaN,...,552746,0,0,1b73fb16c097754ce2024af0e49f9d9514248af0,0,0,0,0,0,NaN
2020-10-23,AL,180916.0,25001.0,1133858.0,NaN,totalTestsViral,1289773.0,888.0,19595.0,NaN,...,1314774,16,147,5323772280706f0dd467c842b6640d6f931d3c5a,0,0,0,0,0,NaN
2020-10-23,AR,104135.0,6796.0,1170162.0,NaN,totalTestsViral,1267501.0,610.0,6678.0,237.0,...,1274297,10,152,678aecb11580eec0d6282ac63a5438e7d88d82c1,0,0,0,0,0,NaN
2020-10-23,AS,0.0,NaN,1616.0,NaN,totalTestsViral,1616.0,NaN,NaN,NaN,...,1616,0,0,48257900fad58cf22cdad9759edd7311027a9b3b,0,0,0,0,0,NaN
2020-10-23,AZ,235882.0,5394.0,1450981.0,NaN,totalTestsPeopleViral,1681469.0,815.0,20967.0,172.0,...,1686863,6,29,fa8d4a28172c518fd0235a20732b7759a3c70bdd,0,0,0,0,0,NaN
